In [ ]:
# # 구글 드라이브 연결
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#this command might be relevant ony on GPU nodes. But the notebook will run on CPU as well
#If running on CPU remove the command wherever it appears
# !nvidia-smi

In [1]:
!apt-get install default-jre
!java -version

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
java version "22.0.2" 2024-07-16
Java(TM) SE Runtime Environment (build 22.0.2+9-70)
Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)


In [2]:
!pip install h2o

In [3]:
import h2o

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)
  Starting server from C:\Users\great\miniconda3\envs\mypjt1\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\great\AppData\Local\Temp\tmpc9jwam59
  JVM stdout: C:\Users\great\AppData\Local\Temp\tmpc9jwam59\h2o_great_started_from_python.out
  JVM stderr: C:\Users\great\AppData\Local\Temp\tmpc9jwam59\h2o_great_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_great_gs6lb6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.814 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [5]:
from h2o.automl import H2OAutoML

In [7]:
# 두번째부터 여기부터 실행하기 train,test 는 전처리가 끝났음, 바로 모델에 학습시키면 됨
import pandas as pd
train = pd.read_csv('./open/train_preprocessed.csv')
train = train.iloc[:,1:]
test = pd.read_csv('./open/test_preprocessed.csv')
test = test.iloc[:,1:]

In [8]:
from sklearn.preprocessing import LabelEncoder
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

In [9]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TRAIN_0002,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAIN_0004,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test.head()

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TEST_0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TEST_0001,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,30,0,0,0,0
2,TEST_0002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TEST_0003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TEST_0004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 1. 데이터 준비
import pandas as pd
from sklearn.model_selection import train_test_split
train_df = h2o.H2OFrame(train)
x_train,x_test,x_valid = train_df.split_frame(ratios=[.7, .15])

y = 'SUBCLASS'
x = x_train.columns
x.remove(y)
x.remove('ID')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
aml = H2OAutoML(max_models = 10, seed = 10, verbosity="info", nfolds=0) # exclude_algos = ["StackedEnsemble", "DeepLearning"]


In [ ]:
aml.train(x = x, y = y, training_frame = x_train, validation_frame=x_valid)

AutoML progress: |
10:32:58.555: Project: AutoML_1_20241009_103258
10:32:58.556: Cross-validation disabled by user: no fold column nor nfolds > 1.
10:32:58.561: Setting stopping tolerance adaptively based on the training frame: 0.015135886972883327
10:32:58.561: Build control seed: 10
10:32:58.575: training frame: Frame key: AutoML_1_20241009_103258_training_py_2_sid_bc90    cols: 4227    rows: 4365  chunks: 13    size: 4701768  checksum: -8920264057157583228
10:32:58.586: validation frame: Frame key: py_4_sid_bc90    cols: 4227    rows: 957  chunks: 13    size: 4435376  checksum: -8926561565497215900
10:32:58.593: leaderboard frame: Frame key: py_4_sid_bc90    cols: 4227    rows: 957  chunks: 13    size: 4435376  checksum: -8926561565497215900
10:32:58.593: blending frame: NULL
10:32:58.593: response column: SUBCLASS
10:32:58.593: fold column: null
10:32:58.593: weights column: null
10:32:58.610: AutoML: XGBoost is not available; skipping it.
10:32:58.615: Loading execution steps: [{X

In [ ]:
test = test.drop(columns=['ID'])
test_filled = test.fillna(0)
test_df = h2o.H2OFrame(test_filled)
predictions = aml.leader.predict(test_df)

In [ ]:
original_labels = le_subclass.inverse_transform(predictions)

In [ ]:
submisson = pd.read_csv("./sample_submission.csv")
submisson["SUBCLASS"] = original_labels
submisson.to_csv('./amino3_aml3.csv', encoding='UTF-8-sig', index=False)